In [1]:
import pandas as pd
import numpy as np

In [3]:
# Loading original data
raw_data = pd.read_csv('Datasets_MS_Project/Pesticides_Trade/Pesticides_Trade_E_All_Data_(Normalized)/Inputs_Pesticides_Trade_E_All_Data_(Normalized).csv')
raw_data.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf4 in position 23849: invalid continuation byte

There is an encoding error. Let's use chardet library to correctly indentify the file encoding first

In [5]:
import chardet

# Read a small sample
with open('Datasets_MS_Project/Pesticides_Trade/Pesticides_Trade_E_All_Data_(Normalized)/Inputs_Pesticides_Trade_E_All_Data_(Normalized).csv', 'rb') as f:
    sample = f.read(10000)

print(chardet.detect(sample))

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


ASCII is a subset of UTF-8. So, they are compatible. Means there are some non-ASCII 
bytes in the dataset. Let's force reading with latin1 encoding which can tolerate 
those non-standard bytes.

In [7]:
raw_data = pd.read_csv('Datasets_MS_Project/Pesticides_Trade/Pesticides_Trade_E_All_Data_(Normalized)/Inputs_Pesticides_Trade_E_All_Data_(Normalized).csv', encoding = 'latin1')
raw_data.head()

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,3,'008,Albania,1357,Pesticides (total),5610,Import quantity,1990,1990,t,1975.862,I
1,3,'008,Albania,1357,Pesticides (total),5610,Import quantity,1991,1991,t,1975.862,I
2,3,'008,Albania,1357,Pesticides (total),5610,Import quantity,1992,1992,t,1975.862,I
3,3,'008,Albania,1357,Pesticides (total),5610,Import quantity,1993,1993,t,1975.862,I
4,3,'008,Albania,1357,Pesticides (total),5610,Import quantity,1994,1994,t,1975.862,I


In [8]:
raw_data.shape

(183505, 12)

In [9]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183505 entries, 0 to 183504
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Area Code        183505 non-null  int64  
 1   Area Code (M49)  183505 non-null  object 
 2   Area             183505 non-null  object 
 3   Item Code        183505 non-null  int64  
 4   Item             183505 non-null  object 
 5   Element Code     183505 non-null  int64  
 6   Element          183505 non-null  object 
 7   Year Code        183505 non-null  int64  
 8   Year             183505 non-null  int64  
 9   Unit             183505 non-null  object 
 10  Value            183505 non-null  float64
 11  Flag             183505 non-null  object 
dtypes: float64(1), int64(5), object(6)
memory usage: 16.8+ MB


In [10]:
# Finding unique values of Item column
raw_data['Item'].unique()

array(['Pesticides (total)', 'Insecticides', 'Fungicides', 'Herbicides',
       'Disinfectants, etc', 'Insecticides (excl. Haz. pest.)',
       'Fungicides (excl. Haz. pest.)', 'Herbicides (excl. Haz. pest.)',
       'Disinfectants, etc (excl. Haz. pest.)', 'Hazardous pesticides',
       'Antimalarial insecticides', 'Alachlor',
       'Aldicarb, captafol & methamidophos',
       'Aldrin, Chlordane, Heptachlor', 'Azinphos-methyl',
       'Compounds of mercury chemically defined, excluding amalgams',
       'DDT, Hexachlorobenzene', 'Dieldrin', 'Dinoseb and its salts',
       'DNOC and its salts', 'Endosulfan',
       'Ethylene dibromide (1,2-dibromoethane)', 'Ethylene dichloride',
       'Fluoroacetamide, monocrotophos & phosphamidon',
       'HCH (mixed isomers) / Lindane',
       'Mercury compounds etc. excl. amalgams',
       'Oxirane (ethylene oxide)', 'Parathion & parathion-methyl',
       'Salts of Pentachlorophenol', 'DDT (clofenotane (INN))',
       '2,4,5-T and its salts and es

In [11]:
raw_data['Element'].unique()

array(['Import quantity', 'Import value', 'Export quantity',
       'Export value'], dtype=object)

For our analysis, we would just consider total pesticides from the list of items 
as we are not going into the pesticide related data analysis. And, from the elements we would 
consider all four of them. 

In [15]:
# Selecting rows where values of Item column equals 'Pesticides (total)'
filtered_data = raw_data.loc[raw_data['Item']=='Pesticides (total)', ]
filtered_data.head()

,Area Code,Area Code (M49),Area,Item Code,Item,Element Code,Element,Year Code,Year,Unit,Value,Flag
0,3,'008,Albania,1357,Pesticides (total),5610,Import quantity,1990,1990,t,1975.862,I
1,3,'008,Albania,1357,Pesticides (total),5610,Import quantity,1991,1991,t,1975.862,I
2,3,'008,Albania,1357,Pesticides (total),5610,Import quantity,1992,1992,t,1975.862,I
3,3,'008,Albania,1357,Pesticides (total),5610,Import quantity,1993,1993,t,1975.862,I
4,3,'008,Albania,1357,Pesticides (total),5610,Import quantity,1994,1994,t,1975.862,I


In [16]:
# Reshaping the dataset into wide-format where each Element has its own column
pivoted_data = filtered_data.pivot_table(
    index = ['Area Code', 'Area', 'Year Code', 'Year'],
    columns = 'Element',
    values = 'Value'
)
pivoted_data.head()

Element                           Export quantity  Export value  \
Area Code Area    Year Code Year                                  
1         Armenia 1992      1992            3.604         3.687   
                  1993      1993            3.604         3.687   
                  1994      1994            3.604         3.687   
                  1995      1995            3.604         3.687   
                  1996      1996            3.604         3.687   

Element                           Import quantity  Import value  
Area Code Area    Year Code Year                                 
1         Armenia 1992      1992          117.861       298.379  
                  1993      1993          117.861       298.379  
                  1994      1994          117.861       298.379  
                  1995      1995          117.861       298.379  
                  1996      1996          117.861       298.379

In [17]:
# Resetting row index
pivoted_data = pivoted_data.reset_index()
pivoted_data.head()

Element,Area Code,Area,Year Code,Year,Export quantity,Export value,Import quantity,Import value
0,1,Armenia,1992,1992,3.604,3.687,117.861,298.379
1,1,Armenia,1993,1993,3.604,3.687,117.861,298.379
2,1,Armenia,1994,1994,3.604,3.687,117.861,298.379
3,1,Armenia,1995,1995,3.604,3.687,117.861,298.379
4,1,Armenia,1996,1996,3.604,3.687,117.861,298.379


In [18]:
# Assigning column index to None
pivoted_data.columns.name = None
pivoted_data.head()

,Area Code,Area,Year Code,Year,Export quantity,Export value,Import quantity,Import value
0,1,Armenia,1992,1992,3.604,3.687,117.861,298.379
1,1,Armenia,1993,1993,3.604,3.687,117.861,298.379
2,1,Armenia,1994,1994,3.604,3.687,117.861,298.379
3,1,Armenia,1995,1995,3.604,3.687,117.861,298.379
4,1,Armenia,1996,1996,3.604,3.687,117.861,298.379


In [19]:
# Renaming the columns
cleaned_data = pivoted_data.rename(
    columns = {
        'Area Code': 'area_code',
        'Area': 'area',
        'Year Code': 'year_code',
        'Year': 'year',
        'Export quantity': 'total_pesticide_export_quantity',
        'Export value': 'total_pesticide_export_value',
        'Import quantity': 'total_pesticide_import_quantity',
        'Import value': 'total_pesticide_import_value'
    }
)
cleaned_data.head()

,area_code,area,year_code,year,total_pesticide_export_quantity,total_pesticide_export_value,total_pesticide_import_quantity,total_pesticide_import_value
0,1,Armenia,1992,1992,3.604,3.687,117.861,298.379
1,1,Armenia,1993,1993,3.604,3.687,117.861,298.379
2,1,Armenia,1994,1994,3.604,3.687,117.861,298.379
3,1,Armenia,1995,1995,3.604,3.687,117.861,298.379
4,1,Armenia,1996,1996,3.604,3.687,117.861,298.379


In [ ]:
# Exporting cleaned data to a csv file 
cleaned_data.to_csv('cleaned_datasets/pesticides_trade_cleaned.csv', index=False, encoding='utf-8')